# Impact assessment

In [1]:
from mescal import *
import pandas as pd
import bw2data as bd

In [2]:
bd.projects.set_current("ecoinvent3.9.1")

In [3]:
esm_location = 'CH'

In [4]:
mapping = pd.read_csv(f'energyscope_data/{esm_location}/mapping_3.9.1_linked.csv')
technology_compositions = pd.read_csv(f'energyscope_data/{esm_location}/technology_compositions.csv')
unit_conversion = pd.read_csv(f'energyscope_data/{esm_location}/unit_conversion_3.9.1.csv')
lifetime = pd.read_csv(f'energyscope_data/{esm_location}/lifetime.csv')
mapping_esm_flows_to_CPC = pd.read_csv(f'energyscope_data/{esm_location}/mapping_esm_flows_to_CPC.csv')

In [5]:
double_counting_removal = pd.read_csv(f'../examples/results/energyscope_{esm_location}/2050/double_counting_removal.csv')
model = pd.read_csv(f'../dev/energyscope_data/{esm_location}/model.csv')
efficiency = pd.read_csv(f'../dev/energyscope_data/{esm_location}/efficiency.csv')
removed_flows = pd.read_csv(f'../examples/results/energyscope_{esm_location}/2050/removed_flows_list.csv')

In [6]:
esm_db = load_extract_db(f'energyscope_{esm_location}_2050')

Getting activity data


100%|██████████| 7285/7285 [00:00<00:00, 232929.85it/s]


Adding exchange data to activities


100%|██████████| 92756/92756 [00:02<00:00, 32750.47it/s]


Filling out exchange data


100%|██████████| 7285/7285 [00:00<00:00, 7822.02it/s] 


In [7]:
db = load_multiple_databases(list(mapping.Database.unique()))

## Efficiencies

In [8]:
esm_db_updated = correct_esm_and_lca_efficiency_differences(
    db=esm_db+db,
    model=model,
    efficiency=efficiency,
    removed_flows=removed_flows,
    unit_conversion=unit_conversion,
    mapping_esm_flows_to_CPC=mapping_esm_flows_to_CPC,
    double_counting_removal=double_counting_removal,
)

No flow found for type WASTE in DHN_COGEN_WASTE. This technology will thus be removed.
No flow found for type WASTE in IND_BOILER_WASTE. This technology will thus be removed.
No flow found for type WASTE in IND_COGEN_WASTE. This technology will thus be removed.
No flow found for type WOOD in PYROLYSIS. This technology will thus be removed.
No flow found for type NG_HP in TRAIN_FREIGHT_NG. This technology will thus be removed.
No flow found for type NG_HP in TRAIN_NG. This technology will thus be removed.
Several names possible for the same type of flow in DEC_BOILER_WOOD: {'wood ash mixture', 'biomass'}. Kept the first one.
Several names possible for the same type of flow in DEC_COGEN_WOOD: {'wood ash mixture', 'biomass'}. Kept the first one.
Several names possible for the same type of flow in DHN_BOILER_WOOD: {'wood ash mixture', 'biomass'}. Kept the first one.
Several names possible for the same type of flow in DHN_COGEN_WOOD: {'wood ash mixture', 'biomass'}. Kept the first one.
Seve

## Compute LCIA metrics

In [5]:
R_long = compute_impact_scores(
    esm_db=esm_db,
    mapping=mapping,
    technology_compositions=technology_compositions,
    methods=['IMPACT World+ Midpoint 2.0.1', 'IMPACT World+ Damage 2.0.1', 'IMPACT World+ Footprint 2.0.1'],
    unit_conversion=unit_conversion,
    lifetime=lifetime
)

In [6]:
R_long.head()

In [7]:
R_long.to_csv(f'energyscope_data/results/{esm_location}/2050/impact_scores.csv', index=False)